In [ ]:
import projX
import pyemma
import numpy as np
from matplotlib import pylab as plt
from os.path import exists
%matplotlib notebook

In [ ]:
trajfile = '/group/ag_cmb/scratch/gph82/work/CMB/hierarchical_TICA/bpti-shaw/DESRES-Trajectory-bpti-c-alpha/c-alpha_centered.stride.100.xtc'
if not exists(trajfile):
    trajfile = '../data/'+trajfile.split('/')[-1]
top  = '/group/ag_cmb/scratch/gph82/work/CMB/hierarchical_TICA/bpti-shaw/DESRES-Trajectory-bpti-c-alpha/bpti-c-alpha_centered.pdb'
if not exists(top):
    top = '../data/'+top.split('/')[-1]

Skip this part if you already have an Y.npy file

In [ ]:
try:
    Y = np.load('Y.npy')
except:
    feat = pyemma.coordinates.featurizer(top)
    pairs = feat.pairs(range(feat.topology.n_atoms)[::2])
    feat.add_distances(pairs)
    src  = pyemma.coordinates.source(trajfile, features=feat)
    tica = pyemma.coordinates.tica(src, lag=10, dim=2)
    Y = tica.get_output()[0]
    np.save('Y.npy', Y)


# Look at the -log(PDF) plot and wonder what structures are behind it

In [ ]:
plt.figure(figsize=(10,10))
h, (x,y) = np.histogramdd(Y, bins=50)
plt.contourf(x[:-1], y[:-1], -np.log(h.T), alpha=.50)

# Use projX to generate a regspace sample of the data

In [ ]:
sample, geoms = projX.generate_sample(trajfile, top, Y, 
                                      n_points=200
                                      #keep_all_samples=True,                                      
                                      #n_geom_samples=200,
                                      #verbose=True
                                    )
sample.shape, geoms

# Link the PDF plot with the sampled structures and visually explore the FES 
Click either on the plot or on the widget slidebar: they're connected! 

In [ ]:
# Replot the FES
plt.figure(figsize=(10,10))
h, (x,y) = np.histogramdd(Y, bins=50)
plt.contourf(x[:-1], y[:-1], -np.log(h.T), alpha=.50)
# Create the linked widget
linked_wdg = projX.visualize_sample(sample, 
                              geoms.superpose(geoms[0]),                             
                              plt.gca(), 
                              clear_lines=True,
                              #plot_path=True
                            )
plt.plot(sample[:,0], sample[:,1],' ok', zorder=0)
# Show it
linked_wdg


# Do everything in one step

In [ ]:
from imp import reload
reload(projX)
ax, fig, iwd, sample, geom = projX.visualize_traj(trajfile, top, 'Y.npy', 
                     #return_data=True
                                          )
iwd

In [ ]:
ax
plt.show()

# Use projX to generate paths along the different projections/axis

In [ ]:
paths_dict, idata = projX.generate_paths(trajfile, top, 
                                         'Y.npy', 
                                         #Y, # You can also directly give the data here
                                         n_projs=2,
                                         verbose=False, 
                                        )

# Link the PDF plot with the sampled paths/structures and visually explore the coordinates (separately). 
Click either on the plot or on the widget slidebar: they're connected! You can change the type of path between min_rmsd or min_disp and you can also change the coordinate sampled (0 or 1)


In [ ]:
# Choose the coordinate and the tyep of path
coord = 0
path_type = 'min_rmsd'
#path_type = 'min_disp'
igeom = paths_dict[coord][path_type]["geom"]
ipath = paths_dict[coord][path_type]["proj"]

In [ ]:
plt.figure(figsize=(10,10))
h, (x,y) = np.histogramdd(Y, bins=50)
plt.contourf(x[:-1], y[:-1], -np.log(h.T), alpha=.50)

linked_wdg = projX.visualize_sample(ipath, 
                              igeom.superpose(igeom[0]),                             
                              plt.gca(), 
                              clear_lines=True,
                              plot_path=True
                            )
linked_wdg

# You can even provide your own clustering object...

In [ ]:
clkmeans = pyemma.coordinates.cluster_kmeans(Y, 10)
sample, geoms = projX.generate_sample(trajfile, top, clkmeans, 
                                     n_geom_samples=50, 
                                     keep_all_samples=True # read the doc for this argument
                                    )

In [ ]:
geom.save('10kmeans.xtc')

In [ ]:
plt.figure(figsize=(7,7))
plt.contourf(x[:-1], y[:-1], -np.log(h.T), alpha=.50)
plt.plot(clkmeans.clustercenters[:,0], clkmeans.clustercenters[:,1],' ok')
iwdg = projX.visualize_sample(sample, 
                              geoms.superpose(geoms[0]),                             
                              plt.gca(), 
                              clear_lines=False,
                              #plot_path=True
                            )
iwdg

In [ ]:
# Check
# https://github.com/arose/nglview/issues/518
# https://github.com/arose/nglview/issues/517

# Intefacing with VMD 
This is a WIP

In [ ]:
plt.figure()
plt.contourf(x[:-1], y[:-1], -np.log(h.T), alpha=.50)

In [ ]:
import bmutils
from imp import reload
reload(bmutils)
vmdpipe = bmutils.link_ax_w_pos_2_vmd(plt.gca(), sample, geoms, rep='Trace', 
                           # freetext= 'start_sscache\n'
                           )

In [ ]:
vmdpipe.close()

In [ ]:
vmdpipe.flush()